## Neste notebook, estarei utilizando os modelos pré-treinados na última camada e descongelando o resto de sua estrutura para fazer o resto do treinamento

In [2]:
import torch
import torch.nn as nn
import torchvision

import io

import pandas as pd

import sklearn.metrics as metrics

In [3]:
device = 'cuda'

Carregamos o modelo resnet18 pré-treinado na última camada.

In [7]:
resnet18 = torchvision.models.resnet18(pretrained=True)

in_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(in_features, 2)

resnet18.load_state_dict(torch.load('./Saved Models/resnet18_pretrained_trainedover1folder.pt'))

<All keys matched successfully>

Carregamos as informações referentes ao dataset.

In [8]:
transform = torchvision.transforms.ToTensor()
DeepFakeDataset = torchvision.datasets.ImageFolder('./Faces Dataset/224px/', transform=transform)

Utilizamos um batch_size de 32, caso contrário a GPU não vai ter memória suficiente

In [15]:
from torch.utils.data.sampler import SubsetRandomSampler

percentage_for_train = 0.7

shuffle_indices = torch.randperm(len(DeepFakeDataset))
train_indices = shuffle_indices[:int(percentage_for_train*len(DeepFakeDataset))]
val_indices = shuffle_indices[int(percentage_for_train*len(DeepFakeDataset)):]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_dataloader = torch.utils.data.DataLoader(DeepFakeDataset, batch_size=128, sampler=train_sampler)
test_dataloader = torch.utils.data.DataLoader(DeepFakeDataset, batch_size=128, sampler=val_sampler)

Criamos um loop de treinamento

In [32]:
import torch.nn.functional as F

def training_loop(n_epochs, model, loss_function, optimizer, print_every_percentage = 20):
    model = model.to(device)
    loss_function = loss_function.to(device)
    size = int(len(train_dataloader)/(100/print_every_percentage))
    
    model.train()
    for epoch in range(0, n_epochs):
        loss_sum = 0
        iteration = 0
        print("Beggining epoch {}...".format(epoch+1))
        for images, labels in train_dataloader:
            
            images = images.to(device)
            labels = labels.to(device)
            
            predictions = model(images)
            loss = loss_function(predictions, labels.long())
            loss_sum += loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            iteration += 1
            
            if iteration % size == 0:
                print("{:.2f}%: Loss {:.6f}".format(round(iteration/len(train_dataloader)*100), loss_sum.item()/size))
                loss_sum = 0

In [11]:
def eval_loop(model):
    all_labels = torch.LongTensor([]).cuda()
    all_predictions = torch.LongTensor([]).cuda()
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            predictions = model(images)
            predictions = predictions.max(dim=1)[1]
            all_predictions = torch.cat((all_predictions, predictions))
            all_labels = torch.cat((all_labels, labels))
            
    return all_predictions, all_labels

In [12]:
def createDataFrame_conf_matrix(all_pread, all_labels):
    c_mat = pd.DataFrame(metrics.confusion_matrix(all_labels.cpu().detach().numpy(), all_pred.cpu().detach().numpy()))
    c_mat = c_mat.rename(columns={0: 'FAKE_predicted', 1: "REAL_predicted"}, index={0: 'FAKE', 1: "REAL"})
    rows = [c_mat.loc['FAKE'][0] / c_mat.sum()[0] * 100, c_mat.loc['REAL'][1] / c_mat.sum()[1] * 100]
    columns = [c_mat['FAKE_predicted'][0] / c_mat.sum(axis=1)[0] * 100, c_mat['REAL_predicted'][1] / c_mat.sum(axis=1)[1] * 100]
    accuracy = (c_mat.loc['FAKE'][0] + c_mat.loc['REAL'][1]) / (c_mat.sum(axis=1)[0] + c_mat.sum(axis=1)[1])*100 
    c_mat.loc['Percentage'] = [str(round(rows[0],2)) +' %', str(round(rows[1],2)) +' %']
    c_mat['Percentage'] = [str(round(columns[0],2)) +' %', str(round(columns[1],2)) +' %', str(round(accuracy,2)) +' %']
    return c_mat

Vamos inicialmente validar nosso modelo do jeito que ele está agora, para verificar seu desempenho.

In [13]:
all_pred, all_labels = eval_loop(resnet18)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,11281,917,92.48 %
REAL,5716,1060,15.64 %
Percentage,66.37 %,53.62 %,65.04 %


Não é o melhor dos desempenhos. Vamos congelar novamente o resto da rede e treinar apenas a última camada agora com o dataset mais balanceado.

In [28]:
for param in resnet18.parameters():
    param.requires_grad = False
    
for param in resnet18.fc.parameters():
    param.requires_grad = True
    
loss_function = nn.CrossEntropyLoss(weight=torch.FloatTensor([1.0, 1.5]))    
optimizer = torch.optim.Adam(resnet18.fc.parameters(), lr=1e-4)

training_loop(n_epochs=3, model=resnet18, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
100.00%: Loss 3.323673
Beggining epoch 2...
100.00%: Loss 3.016788
Beggining epoch 3...
100.00%: Loss 2.892854


In [31]:
training_loop(n_epochs=1, model=resnet18, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
19.94%: Loss 0.565882
39.88%: Loss 0.564261
59.83%: Loss 0.567212
79.77%: Loss 0.561800
99.71%: Loss 0.554914


Testamos novamente...

In [33]:
all_pred, all_labels = eval_loop(resnet18)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,9108,3150,74.3 %
REAL,2118,4598,68.46 %
Percentage,81.13 %,59.34 %,72.24 %


Uma melhora substancial, mas ainda longe de perfeita. Vamos agora descongelar o resto da rede, congelar a última camada e treinar mais duas épocas.

In [36]:
for param in resnet18.parameters():
    param.requires_grad = True
    
for param in resnet18.fc.parameters():    
    param.requires_grad = False
    
loss_function = nn.CrossEntropyLoss(weight=torch.FloatTensor([1.0, 1.5]))    
optimizer = torch.optim.Adam(resnet18.parameters(), lr=1e-4)

training_loop(n_epochs=1, model=resnet18, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
20.00%: Loss 0.343564
40.00%: Loss 0.199451
60.00%: Loss 0.165088
80.00%: Loss 0.134616
100.00%: Loss 0.121583


In [37]:
all_pred, all_labels = eval_loop(resnet18)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,12088,170,98.61 %
REAL,766,5950,88.59 %
Percentage,94.04 %,97.22 %,95.07 %


Uma melhora impressionante. Vamos agora descongelar toda a rede e treinar mais uma época com um número menor de batch_size

In [38]:
train_dataloader = torch.utils.data.DataLoader(DeepFakeDataset, batch_size=32, sampler=train_sampler)
test_dataloader = torch.utils.data.DataLoader(DeepFakeDataset, batch_size=32, sampler=val_sampler)

for param in resnet18.parameters():
    param.requires_grad = True
    
loss_function = nn.CrossEntropyLoss(weight=torch.FloatTensor([1.0, 1.5]))    
optimizer = torch.optim.Adam(resnet18.parameters(), lr=1e-4)

training_loop(n_epochs=2, model=resnet18, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
20.00%: Loss 0.158404
40.00%: Loss 0.137011
60.00%: Loss 0.131758
80.00%: Loss 0.114225
100.00%: Loss 0.117139
Beggining epoch 2...
20.00%: Loss 0.076221
40.00%: Loss 0.076924
60.00%: Loss 0.081203
80.00%: Loss 0.082430
100.00%: Loss 0.087860


In [39]:
all_pred, all_labels = eval_loop(resnet18)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,11691,567,95.37 %
REAL,170,6546,97.47 %
Percentage,98.57 %,92.03 %,96.12 %


Excelente!

Podemos melhorar um pouco nosso resultado para a classe de fakes. Vamos diminuir o peso para REAL da função de curto, diminuir o learning rate um pouco e treinar mais uma época. Mas antes vamos salvar por via das dúvidas hehe.

In [40]:
torch.save(resnet18.state_dict(), './Saved Models/resnet18_balanced_unfreezed_F95_R97_A96.pt')

In [41]:
loss_function = nn.CrossEntropyLoss(weight=torch.FloatTensor([1.0, 1.2]))    
optimizer = torch.optim.Adam(resnet18.parameters(), lr=0.7e-4)

training_loop(n_epochs=1, model=resnet18, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
20.00%: Loss 0.047889
40.00%: Loss 0.045736
60.00%: Loss 0.056987
80.00%: Loss 0.048687
100.00%: Loss 0.048200


In [42]:
all_pred, all_labels = eval_loop(resnet18)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,11966,292,97.62 %
REAL,300,6416,95.53 %
Percentage,97.55 %,95.65 %,96.88 %


Conseguimos! Mas agora caímos para a classe de reais haha. Vamos tentar abaixar mais o learning rate e aumentar um pouco o peso para a classe do REAL, afim de achar um balanço melhor. Vamos acrescentar um fator baixo de regularização L2.

In [43]:
loss_function = nn.CrossEntropyLoss(weight=torch.FloatTensor([1.0, 1.35]))
optimizer = torch.optim.Adam(resnet18.parameters(), lr=0.5e-4, weight_decay=1e-4)

training_loop(n_epochs=1, model=resnet18, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
20.00%: Loss 0.029221
40.00%: Loss 0.028134
60.00%: Loss 0.029542
80.00%: Loss 0.027856
100.00%: Loss 0.032376


In [44]:
all_pred, all_labels = eval_loop(resnet18)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,12078,180,98.53 %
REAL,299,6417,95.55 %
Percentage,97.58 %,97.27 %,97.48 %


Melhoramos todos os parâmetros em algum nível. Ótimo! Vamos salvar e tentar retornar o valor do weight para a classe real e treinar mais uma época.

In [45]:
torch.save(resnet18.state_dict(), './Saved Models/resnet18_balanced_unfreezed_F98_R95_A97.pt')

In [46]:
loss_function = nn.CrossEntropyLoss(weight=torch.FloatTensor([1.0, 1.5]))
optimizer = torch.optim.Adam(resnet18.parameters(), lr=0.55e-4, weight_decay=1e-4)

training_loop(n_epochs=1, model=resnet18, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
20.00%: Loss 0.025860
40.00%: Loss 0.020100
60.00%: Loss 0.027787
80.00%: Loss 0.029885
100.00%: Loss 0.026672


In [47]:
all_pred, all_labels = eval_loop(resnet18)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,11942,316,97.42 %
REAL,173,6543,97.42 %
Percentage,98.57 %,95.39 %,97.42 %


Nossa acurácia caiu, mas conseguimos a mesma proporção de acerto para as 2 classes, perfeito! Vamos congelar a rede nesse estado e treinar mais um pouco a camada fc para tentarmos alcançar um resultado um pouco melhor.

In [52]:
torch.save(resnet18.state_dict(), './Saved Models/resnet18_balanced_unfreezed_F97_R97_A97.pt')

In [50]:
for param in resnet18.parameters():
    param.requires_grad = False
    
for param in resnet18.fc.parameters():
    param.requires_grad = True

loss_function = nn.CrossEntropyLoss(weight=torch.FloatTensor([1.0, 1.5]))
optimizer = torch.optim.Adam(resnet18.fc.parameters(), lr=1e-5)

training_loop(n_epochs=2, model=resnet18, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
20.00%: Loss 0.017157
40.00%: Loss 0.015431
60.00%: Loss 0.016324
80.00%: Loss 0.019063
100.00%: Loss 0.015250
Beggining epoch 2...
20.00%: Loss 0.015029
40.00%: Loss 0.016971
60.00%: Loss 0.016239
80.00%: Loss 0.014240
100.00%: Loss 0.015326


In [53]:
optimizer = torch.optim.Adam(resnet18.fc.parameters(), lr=3e-5)
training_loop(n_epochs=1, model=resnet18, loss_function=loss_function, optimizer=optimizer)

Beggining epoch 1...
20.00%: Loss 0.013096
40.00%: Loss 0.018334
60.00%: Loss 0.014609
80.00%: Loss 0.014736
100.00%: Loss 0.013762


In [54]:
all_pred, all_labels = eval_loop(resnet18)
c_mat = createDataFrame_conf_matrix(all_pred, all_labels)
c_mat

,FAKE_predicted,REAL_predicted,Percentage
FAKE,12003,255,97.92 %
REAL,191,6525,97.16 %
Percentage,98.43 %,96.24 %,97.65 %


In [ ]:
# Aqui estou experimentado colocar um peso 9x maior para a classe de não deepfake
loss_function = nn.CrossEntropyLoss(weight=torch.FloatTensor([1.0, 1.5]))

optimizer = torch.optim.Adam(resnet50.fc.parameters(), lr=1e-4)

training_loop(n_epochs=5, model=resnet50, loss_function=loss_function, optimizer=optimizer)